In [9]:
# Function to reduce the size of the network
function reduce_size(eng)

    # Helper function to find lines connected to a bus
    function find_lines_connected_to_bus(eng, bus_id)
        lines_connected = []
        for (line_id, line_info) in eng["line"]
            if line_info["f_bus"] == bus_id || line_info["t_bus"] == bus_id
                push!(lines_connected, (line_id, line_info))
            end
        end
        return lines_connected
    end

    # Iterate over buses to find and remove the target buses
    buses_to_remove = []
    for (bus_id, bus_info) in eng["bus"]
        # Condition 1: Check if the bus has no load connected to it
        load_connected = any(load_info["bus"] == bus_id for load_info in values(eng["load"]))
        if load_connected
            continue
        end

        # Find lines connected to this bus
        lines_connected = find_lines_connected_to_bus(eng, bus_id)

        # Condition 2: Check if the bus only connects two lines
        if length(lines_connected) != 2
            continue
        end

        # Unpack the connected lines
        line1_id, line1_info = lines_connected[1]
        line2_id, line2_info = lines_connected[2]

        # Condition 3: Check if the "line_code" of the two lines is the same
        if line1_info["linecode"] != line2_info["linecode"]
            continue
        end

        # All conditions met, prepare to remove this bus
        push!(buses_to_remove, bus_id)

        # Update the remaining line's information
        new_length = line1_info["length"] + line2_info["length"]

        if line1_info["f_bus"] == line2_info["f_bus"] || line1_info["t_bus"] == line2_info["t_bus"]
            error("f_bus and t_bus in this network don't match line flow direction.")
        end
    
        if line1_info["f_bus"] == bus_id
            new_t_bus = line1_info["t_bus"]
            new_f_bus = line2_info["f_bus"]
            new_t_connection = line1_info["t_connections"]
            new_f_connection = line2_info["f_connections"]
        else
            new_f_bus = line1_info["f_bus"]
            new_t_bus = line2_info["t_bus"]
            new_f_connection = line1_info["f_connections"]
            new_t_connection = line2_info["t_connections"]
        end

        remaining_line_id = line1_id
        eng["line"][remaining_line_id]["length"] = new_length
        eng["line"][remaining_line_id]["f_bus"] = new_f_bus
        eng["line"][remaining_line_id]["t_bus"] = new_t_bus
        eng["line"][remaining_line_id]["f_connections"] = new_f_connection
        eng["line"][remaining_line_id]["t_connections"] = new_t_connection

        # Remove the other line
        delete!(eng["line"], line2_id)
    end

    # Remove the buses
    for bus_id in buses_to_remove
        delete!(eng["bus"], bus_id)
    end

    return eng
end

reduce_size (generic function with 2 methods)

# Function Documentation

## Overview
The `reduce_size` function reduces the size of an electrical network by removing specific buses and updating the connections between lines based on certain conditions. This function is useful for simplifying network models while preserving essential connectivity and properties.

## Inputs
- `eng`: A dictionary representing the electrical network. This dictionary contains information about buses and lines in the network.

## Outputs
- A modified version of the `eng` dictionary with certain buses and lines removed, and the remaining lines updated to maintain network connectivity.

## Functionality
1. **Helper Function**: `find_lines_connected_to_bus(eng, bus_id)` - Finds and returns a list of lines connected to a given bus.
2. **Iterate Over Buses**: The function loops through each bus in the network to identify candidates for removal.
3. **Check Conditions**:
   - The bus has no load connected to it.
   - The bus connects exactly two lines.
   - The two lines have the same "linecode".
4. **Update Line Information**: If all conditions are met, the function:
   - Adds the bus to a removal list.
   - Merges the lengths of the two lines.
   - Updates the `f_bus` and `t_bus` connections for the remaining line.
5. **Remove Redundant Lines and Buses**: The function deletes the redundant line and all marked buses from the network.
6. **Return Modified Network**: The modified network dictionary is returned.
